In [62]:
import pandas as pd
import numpy as np
import os
import json
import re
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tflearn
import nltk
from nltk.stem.lancaster import LancasterStemmer

In [65]:
dfload = pd.read_json(os.getcwd()+'/../Data/convertcsv.json')
df = pd.DataFrame(columns=['TAGS','PATTERNS','RESPONSES'])
i = 0
for _,r in dfload.iterrows():
    for resp in r[0]['response']:
        for p in r[0]['patterns']:
            df.set_value(i, 'PATTERNS', p)
            df.set_value(i, 'TAGS', r[0]['tag'])
            df.set_value(i, 'RESPONSES', resp)
            i += 1
df

F:\anaconda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
F:\anaconda\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
F:\anaconda\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


,TAGS,PATTERNS,RESPONSES
0,greeting,Hi,"Hey there, friend! I'm InvestoBot, the chatbot..."
1,greeting,Hey,"Hey there, friend! I'm InvestoBot, the chatbot..."
2,greeting,Hello,"Hey there, friend! I'm InvestoBot, the chatbot..."
3,greeting,I am fine. And you?,"Hey there, friend! I'm InvestoBot, the chatbot..."
4,greeting,How are you,"Hey there, friend! I'm InvestoBot, the chatbot..."
5,greeting,Is there anyone,"Hey there, friend! I'm InvestoBot, the chatbot..."
6,greeting,"Good Morning, I need some help","Hey there, friend! I'm InvestoBot, the chatbot..."
7,greeting,Howdy !!,"Hey there, friend! I'm InvestoBot, the chatbot..."
8,greeting,"Hey, I have a question regarding investments.","Hey there, friend! I'm InvestoBot, the chatbot..."
9,greeting,"Hello, Good Day ! How are you ?","Hey there, friend! I'm InvestoBot, the chatbot..."


In [66]:
# df = pd.read_csv(os.getcwd()+'/../Data/NLPData.csv')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df

,TAGS,PATTERNS,RESPONSES
0,greeting,Hi,"Hey there, friend! I'm InvestoBot, the chatbot..."
1,greeting,Hey,"Hey there, friend! I'm InvestoBot, the chatbot..."
2,greeting,Hello,"Hey there, friend! I'm InvestoBot, the chatbot..."
3,greeting,I am fine. And you?,"Hey there, friend! I'm InvestoBot, the chatbot..."
4,greeting,How are you,"Hey there, friend! I'm InvestoBot, the chatbot..."
5,greeting,Is there anyone,"Hey there, friend! I'm InvestoBot, the chatbot..."
6,greeting,"Good Morning, I need some help","Hey there, friend! I'm InvestoBot, the chatbot..."
7,greeting,Howdy !!,"Hey there, friend! I'm InvestoBot, the chatbot..."
8,greeting,"Hey, I have a question regarding investments.","Hey there, friend! I'm InvestoBot, the chatbot..."
9,greeting,"Hello, Good Day ! How are you ?","Hey there, friend! I'm InvestoBot, the chatbot..."


In [3]:
# newdf = pd.DataFrame(columns=['TAGS','PATTERNS'])
# index = 0
# for i in range(len(df)):
#     arr = df['PATTERNS'][i].split(',')
#     for a in arr:
#         newdf.set_value(index, 'TAGS', df['TAGS'][i])
#         newdf.set_value(index, 'PATTERNS', a)
#         index += 1
# newdf

F:\anaconda\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
F:\anaconda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


,TAGS,PATTERNS
0,GREETINGS,Hi Hello
1,GREETINGS,How are you
2,GREETINGS,Is there anyone
3,GREETINGS,Good Morning
4,GREETINGS,I need some help
5,GREETINGS,Howdy !!
6,GREETINGS,Hey
7,GREETINGS,I have a question regarding investments
8,GREETINGS,hi
9,GREETINGS,Good Afternoon ! How is your day going?


In [67]:
class PreProcessMessage():

    def __init__(self):
        pass

    def preprocess_word(self, word):
        # Remove punctuation
        word = word.strip('\'"?!,.():;')
        # Convert more than 2 letter repetitions to 2 letter
        # funnnnny --> funny
        word = re.sub(r'(.)\1+', r'\1\1', word)
        # Remove - & '
        word = re.sub(r'(-|\')', '', word)
        return word


    def is_valid_word(self, word):
        # Check if word begins with an alphabet
        return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)


    def handle_emojis(self, message):
        # Smile -- :), : ), :-), (:, ( :, (-:, :')
        message = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', message)
        # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
        message = re.sub(r'(:\s?d|:-d|x-?d|x-?d)', ' EMO_POS ', message)
        # Love -- <3, :*
        message = re.sub(r'(<3|:\*)', ' EMO_POS ', message)
        # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
        message = re.sub(r'(;-?\)|;-?d|\(-?;)', ' EMO_POS ', message)
        # Sad -- :-(, : (, :(, ):, )-:
        message = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', message)
        # Cry -- :,(, :'(, :"(
        message = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', message)
        return message

    def preprocess_message(self, message):
        processed_message = []
        # Convert to lower case
        message = message.lower()
        # Replaces URLs with the word URL
        message = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', message)
        # Replace 2+ dots with space
        message = re.sub(r'\.{2,}', ' ', message)
        # Strip space, " and ' from message
        message = message.strip(' "\'')
        # Replace emojis with either EMO_POS or EMO_NEG
        message = self.handle_emojis(message)
        # Replace multiple spaces with a single space
        message = re.sub(r'\s+', ' ', message)
        words = message.split()

        for word in words:
            word = self.preprocess_word(word)
            if self.is_valid_word(word):
                processed_message.append(word)
        return ' '.join(processed_message)

    def transformCleanedMessage(self, message):
        tfv=TfidfVectorizer(min_df=0, max_features=None, strip_accents='unicode',lowercase =True,
        analyzer='word', token_pattern=r'\w{3,}', ngram_range=(1,1), sublinear_tf=True, stop_words = "english")
        transformedMessage=tfv.fit_transform(message)
        return transformedMessage

In [68]:
newdf = df
newdf.TAGS.unique()

array(['greeting', 'name', 'yesAccount', 'noAccount',
       'accountNumberEntered', 'yesPortfolioOptimize',
       'noPortfolioOptimize', 'yesInvestment', 'noInvestment',
       'charityInvestment', 'educationInvestment', 'retirementInvestment',
       'personalInvestment', 'foreignExchangeInvestment',
       'bondsInvestment', 'mutualFundsInvestment', 'stocksInvestment',
       'yesStockInvestment', 'noStockInvestment', 'exit'], dtype=object)

In [69]:
newdf.TAGS = pd.Categorical(newdf.TAGS)
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 3 columns):
TAGS         129 non-null category
PATTERNS     129 non-null object
RESPONSES    129 non-null object
dtypes: category(1), object(2)
memory usage: 3.0+ KB


In [70]:
le = LabelEncoder()
newdf['Target'] = le.fit_transform(newdf.TAGS)
newdf

,TAGS,PATTERNS,RESPONSES,Target
0,greeting,Hi,"Hey there, friend! I'm InvestoBot, the chatbot...",6
1,greeting,Hey,"Hey there, friend! I'm InvestoBot, the chatbot...",6
2,greeting,Hello,"Hey there, friend! I'm InvestoBot, the chatbot...",6
3,greeting,I am fine. And you?,"Hey there, friend! I'm InvestoBot, the chatbot...",6
4,greeting,How are you,"Hey there, friend! I'm InvestoBot, the chatbot...",6
5,greeting,Is there anyone,"Hey there, friend! I'm InvestoBot, the chatbot...",6
6,greeting,"Good Morning, I need some help","Hey there, friend! I'm InvestoBot, the chatbot...",6
7,greeting,Howdy !!,"Hey there, friend! I'm InvestoBot, the chatbot...",6
8,greeting,"Hey, I have a question regarding investments.","Hey there, friend! I'm InvestoBot, the chatbot...",6
9,greeting,"Hello, Good Day ! How are you ?","Hey there, friend! I'm InvestoBot, the chatbot...",6


In [71]:
processed_set = []
only_text = pd.DataFrame(newdf['PATTERNS'])
for i in range(0, len(newdf)):
    processed = PreProcessMessage().preprocess_message(only_text['PATTERNS'][i])
    processed_set.append(processed)

In [72]:
X_train, X_test, y_train, y_test = train_test_split(processed_set,newdf.Target, test_size=0.2, random_state=42)

In [73]:
tfv=TfidfVectorizer(min_df=0, max_features=None, strip_accents='unicode',lowercase =True,
        analyzer='word', token_pattern=r'\w{3,}', ngram_range=(1,1), sublinear_tf=True, stop_words = "english")
X_train_transformed=tfv.fit_transform(X_train)
X_test_transformed=tfv.transform(X_test)
processed_set_transformed = tfv.fit_transform(processed_set)

In [74]:
model=LogisticRegression(multi_class='ovr')
model.fit(processed_set_transformed,newdf.Target)
model.score(processed_set_transformed, newdf.Target)

F:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7829457364341085

## XGBoost

In [75]:
from xgboost import XGBClassifier
xgb = XGBClassifier(objective='multi:softmax', num_class=newdf.Target.nunique)
xgb.fit(processed_set_transformed,newdf.Target)
xgb.score(processed_set_transformed, newdf.Target)

0.7751937984496124

## RandomForest

In [76]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(processed_set_transformed,newdf.Target)
rf.score(processed_set_transformed, newdf.Target)

F:\anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9612403100775194

## Neural Nets

In [90]:
with open(os.getcwd()+'/../Data/convertcsv.json') as file:
    data = json.load(file)
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)

In [91]:
tf.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 16)
net = tflearn.fully_connected(net, 16)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)

Training Step: 17999  | total loss: 1.77092 | time: 0.060s
| Adam | epoch: 1000 | loss: 1.77092 - acc: 0.9125 -- iter: 136/139
Training Step: 18000  | total loss: 1.59386 | time: 0.067s
| Adam | epoch: 1000 | loss: 1.59386 - acc: 0.9212 -- iter: 139/139
--


In [92]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)

results = model.predict([bag_of_words("Hey i am gaurang", words)])[0]
results_index = np.argmax(results)
tag = labels[results_index]
print(tag)
if results[results_index] < 0.7:
    print('I don\'t know')
    print(results[results_index])

yes
I don't know
0.46907035


In [93]:
with open(os.getcwd()+"/../models/data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

In [94]:
model.save(os.getcwd()+'/../models/model.tflearn')

INFO:tensorflow:F:\anaconda\Scripts\dsemt\project\InvestoBot\Training/../models/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [56]:
model.load('model.tflearn')

INFO:tensorflow:Restoring parameters from F:\anaconda\Scripts\dsemt\project\InvestoBot\Training\model.tflearn


In [82]:
output.shape

(132, 22)